In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.6/720.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 22.2 MB/s eta 0:00:00


In [1]:
import torch
import torch.nn as nn

import torch.nn.functional as F
import torch.optim as optim
import torchmetrics
import torchvision

import pytorch_lightning as pl


import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split



In [10]:
from typing import Any


class GraphAttentionModulation_LINEAR(nn.Module):
    def __init__(self, dim_embedding, max_num_boxes, temperature = 1.0) -> None:
        super().__init__()

        self.dim_embedding = dim_embedding
        self.num_boxes = max_num_boxes
        self.temperature = temperature

        self.W = nn.Bilinear(max_num_boxes, dim_embedding, dim_embedding)

        self.W1 = nn.Linear(dim_embedding, dim_embedding)

        self.gelu = nn.GELU()

        self.layer_norm = nn.LayerNorm(dim_embedding)

    def forward(self, x):

        # compute the clip score
        # x: (batch_size, n_boxes, dim_embedding)
        # c_s: (batch_size, n_boxes, n_boxes)
        # the clip score is the similarity between each pair of boxes
        # c_s[i, j] is the similarity between box i and box j
        # the similarity will be used to compute the attention score
        # considering the similarity between each pair of boxes
        # as the link in between the boxes

        # normalize the embedding
        # x /= x.norm(dim = -1, keepdim = True)
        # c_s = F.softmax(torch.bmm(x, x.permute(0, 2, 1)) / (2.0 * self.temperature), dim=-1)
        c_s = F.softmax(torch.bmm(x, x.permute(0, 2, 1)), dim=-1)
        # c_s = torch.bmm(x, x.permute(0, 2, 1))

        # find a transformation of the embedding
        # based on the similarity between each pair of boxes
        # c_s W^T -> attention_score
        # attention_score: (batch_size, n_boxes, n_boxes)
        # attention_score Embedding -> (batch_size, n_boxes, dim_embedding)
        x = self.W(c_s, x)
        del c_s

        # apply a non-linear transformation
        x = self.gelu(x)

        # apply a layer normalization
        x = self.layer_norm(x)

        # apply a linear transformation
        x = self.W1(x)

        # apply a non-linear transformation
        x = self.gelu(x)

        # apply a layer normalization
        x = self.layer_norm(x)

        return x

##############################################################################
##############################################################################

class GraphBoxRegressor(nn.Module):
    def __init__(self, dim_embedding = 512, max_num_boxes = 48) -> None:
        super().__init__()

        self.dim_embedding = dim_embedding
        self.max_num_boxes = max_num_boxes

        self.gam = GraphAttentionModulation_LINEAR(dim_embedding, max_num_boxes)

        self.flatten = nn.Flatten(start_dim=1, end_dim=- 1)

        self.regressor = nn.Sequential(
            # nn.Linear(dim_embedding * max_num_boxes + max_num_boxes * 4, dim_embedding),
            nn.Linear(1 * max_num_boxes + max_num_boxes * 4, dim_embedding),
            nn.GELU(),
            nn.LayerNorm(dim_embedding),
            nn.Linear(dim_embedding, dim_embedding//2),
            nn.GELU(),
            nn.LayerNorm(dim_embedding//2),
            nn.Linear(dim_embedding//2, 4)
        )

    def forward(self, text_feat, x, boxes):

        # apply the graph attention modulation
        # x: (batch_size, n_boxes, dim_embedding)
        x = self.gam(x)

        # x /= x.norm(dim = -1, keepdim = True)
        # text_feat /= text_feat.norm(dim = -1, keepdim = True)

        x = torch.bmm(x, text_feat)

        # print(x.shape, boxes.shape)

        # concatenate the boxes to the embedding
        # x: (batch_size, n_boxes, 1 + 4)
        x = torch.cat([x, boxes], dim = -1)

        # flatten the embedding
        # x: (batch_size, n_boxes * dim_embedding)
        x = self.flatten(x)

        # apply the regressor
        # x: (batch_size, 4)
        x = self.regressor(x)

        return x


##############################################################################
##############################################################################


class GraphBoxRegressorLightning(pl.LightningModule):
    def __init__(self, dim_embedding = 512, max_num_boxes = 48) -> None:
        super().__init__()

        self.dim_embedding = dim_embedding
        self.max_num_boxes = max_num_boxes

        self.gam_l = GraphBoxRegressor(dim_embedding, max_num_boxes)


        self.MSE = nn.MSELoss()
        self.MAE = nn.L1Loss()
        self.HUBER = nn.SmoothL1Loss()

        # GENERALIZED_BOX_IOU_LOSS https://arxiv.org/abs/1902.09630
        self.generalized_box_iou_loss = torchvision.ops.generalized_box_iou_loss
        # DISTANCE_BOX_IOU_LOSS https://arxiv.org/abs/1911.08287
        self.distance_box_iou_loss = torchvision.ops.distance_box_iou_loss
        # COMPLETE_BOX_IOU_LOSS https://arxiv.org/abs/1911.08287
        self.complete_box_iou_loss = torchvision.ops.complete_box_iou_loss


    def forward(self,text_feat, x, boxes):
        return self.gam_l(text_feat, x, boxes)

    def training_step(self, batch, batch_idx):

        text_feat, x, boxes, target = batch

        pred = self(text_feat, x, boxes)

        # keep track of the losses

        mse_loss = self.MSE(pred, target)

        mae_loss = self.MAE(pred, target)

        huber_loss = self.HUBER(pred, target)

        g_box_iou_loss = abs(self.generalized_box_iou_loss(pred, target, reduction = 'mean'))

        d_box_iou_loss = abs(self.distance_box_iou_loss(pred, target, reduction = 'mean'))

        c_box_iou_loss = abs(self.complete_box_iou_loss(pred, target, reduction = 'mean'))

        self.log('train_mse_loss', mse_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('train_mae_loss', mae_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('train_huber_loss', huber_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('train_g_box_iou_loss', g_box_iou_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('train_d_box_iou_loss', d_box_iou_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('train_c_box_iou_loss', c_box_iou_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)

        return huber_loss

    def validation_step(self, batch, batch_idx):

        text_feat, x, boxes, target = batch

        pred = self(text_feat, x, boxes)

        # keep track of the losses

        mse_loss = self.MSE(pred, target)

        mae_loss = self.MAE(pred, target)

        huber_loss = self.HUBER(pred, target)

        g_box_iou_loss = abs(self.generalized_box_iou_loss(pred, target, reduction = 'mean'))

        d_box_iou_loss = abs(self.distance_box_iou_loss(pred, target, reduction = 'mean'))

        c_box_iou_loss = abs(self.complete_box_iou_loss(pred, target, reduction = 'mean'))

        self.log('val_mse_loss', mse_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('val_mae_loss', mae_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('val_huber_loss', huber_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('val_g_box_iou_loss', g_box_iou_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('val_d_box_iou_loss', d_box_iou_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('val_c_box_iou_loss', c_box_iou_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)

        return huber_loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr = 1e-3)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 10)
        return [optimizer], [scheduler]





In [3]:
# x = torch.randn(64, 48, 512)
# text_feat = torch.randn(64, 512, 1)
# boxes = torch.randn(64, 48, 4)

# x = torch.bmm(x, text_feat)

# torch.cat([x, boxes], dim = -1).shape

Code to retrieve the precomputed data. Then, here is done futher proeprocessing to organize the data into batches.

In [4]:
import pickle
import torch

# load test dtaset
file_name = '/content/drive/MyDrive/refCOCOg Visual Grounding/yolov8x+clip/yolo_v8x_1_dictionary_full_train.p'
with open(file_name, 'rb') as f:
    data_train = pickle.load(f)

# load test dataset
# file_name = '/content/drive/MyDrive/refCOCOg Visual Grounding/yolov8x+clip/yolo_v8x_1_dictionary_full_test.p'
# with open(file_name, 'rb') as f:
#     data_test = pickle.load(f)

# load test dataset
file_name = '/content/drive/MyDrive/refCOCOg Visual Grounding/yolov8x+clip/yolo_v8x_1_dictionary_full_val.p'
with open(file_name, 'rb') as f:
    data_val = pickle.load(f)



from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

def get_data(full_data, max_sample = 20000):

    text_encoding, box_encoding, box_coords, target_boxes = [], [], [], []

    for idx in tqdm(list(full_data)[:max_sample]):
        # for _ in range(data['image_emb'].shape[0]):
        for idx_text in range(full_data[idx]['text_emb'].shape[0]):

            # number of available crops
            number_of_crop = min(full_data[idx]['image_emb'].shape[0], len(full_data[idx]['df_boxes']))

            if number_of_crop == 0:
                break

            # shape: (number of samples, 512, 1)
            text_encoding.append(full_data[idx]['text_emb'][idx_text].unsqueeze(1))

            number_of_crop = min(full_data[idx]['image_emb'].shape[0], len(full_data[idx]['df_boxes']))

            # shape: (number of samples, 512, number of crop embeddings)
            box_encoding.append(full_data[idx]['image_emb'][:number_of_crop,:].permute(1, 0))

            # shape: (number of samples, number of boxes, 4)
            box_coords.append(torch.stack([torch.tensor(full_data[idx]['df_boxes'].iloc[i][:4]).type(torch.float16)
                                                    for i in range(number_of_crop)]))

            # shape: (number of samples, 1, 4)
            target_boxes.append(torch.tensor(full_data[idx]['bbox_target']).type(torch.float16).unsqueeze(0))

    return torch.stack(text_encoding), torch.stack([torch.nn.functional.pad(b.permute(1, 0), (0, 0, 0, 48 - b.shape[1])).permute(1, 0) for b in box_encoding]), torch.stack([torch.nn.functional.pad(b, (0, 0, 0, 48 - b.shape[0])) for b in box_coords]), torch.stack(target_boxes)



# text_encoding_test, box_encoding_test, box_coords_test, target_boxes_test = get_data(data_test)
text_encoding_val, box_encoding_val, box_coords_val, target_boxes_val = get_data(data_val)
del data_val
text_encoding_train, box_encoding_train, box_coords_train, target_boxes_train = get_data(data_train)
del data_train
# text_encoding_train, box_encoding_train, box_coords_train, target_boxes_train = text_encoding_train[:20000], box_encoding_train[:20000], box_coords_train[:20000], target_boxes_train[:20000]


def box_norm_rescale(box_target):
    """ Rescale the box_target
    Args:
        box_target: (number of samples, 1, 4)

    Returns:
        box_target: (number of samples, 1, 4)

    """
    # convert the box_pred to x1, y1, x2, y2
    box_target[:, 0, 2] = box_target[:, 0, 0] + box_target[:, 0, 2]
    box_target[:, 0, 3] = box_target[:, 0, 1] + box_target[:, 0, 3]

    return box_target

# box rescaling
# target_boxes_test = box_norm_rescale(target_boxes_test)
target_boxes_val = box_norm_rescale(target_boxes_val)
target_boxes_train = box_norm_rescale(target_boxes_train)


100%|██████████| 20000/20000 [00:51<00:00, 387.70it/s]


In [5]:
text_encoding_train.shape, box_encoding_train.shape, box_coords_train.shape, target_boxes_train.shape

(torch.Size([38295, 512, 1]),
 torch.Size([38295, 512, 48]),
 torch.Size([38295, 48, 4]),
 torch.Size([38295, 1, 4]))

In [6]:
target_boxes_train = target_boxes_train.squeeze(1)
target_boxes_val = target_boxes_val.squeeze(1)

# set up the dataloaders
BATCH_SIZE = 256


train_dataset = torch.utils.data.TensorDataset(text_encoding_train.type(torch.float32), box_encoding_train.permute(0, 2, 1).type(torch.float32), box_coords_train.type(torch.float32), target_boxes_train.type(torch.float32))
val_dataset = torch.utils.data.TensorDataset(text_encoding_val.type(torch.float32), box_encoding_val.permute(0, 2, 1).type(torch.float32), box_coords_val.type(torch.float32), target_boxes_val.type(torch.float32))


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = False)


# train_size = int(len(X)*0.8)
# val_size = len(X) - train_size
# dataset = torch.utils.data.TensorDataset(X, boxes, target)
# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])


In [14]:
torch.autograd.set_detect_anomaly(True)

In [9]:
# set up the model

model = GraphBoxRegressorLightning().cuda()

# early stopping
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

early_stop_callback = EarlyStopping('val_g_box_iou_loss', patience = 70, mode = 'min')
checkpoint_callback = ModelCheckpoint(monitor = 'val_g_box_iou_loss', save_top_k = 1, mode = 'min', dirpath='/content/drive/MyDrive/Colab Notebooks/dl/graphattention/', filename='model-gam_l-{epoch:02d}-{val_loss:.2f}')
# set up the trainer

trainer = pl.Trainer(accelerator='auto', max_epochs = 200 , callbacks=[early_stop_callback, checkpoint_callback])

# train the model
trainer.fit(model, train_loader, val_loader)



INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /content/drive/MyDrive/Colab Notebooks/dl/graphattention exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type              | Params
--------------------------------------------
0 | gam_l | GraphBoxRegressor | 13.1 M
1 | MSE   | MSELoss           | 0     
2 | MAE   | L1Loss            | 0     
3 | HUBER | SmoothL1Loss      | 0     
-------------------

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [11]:
# set up the model

model = GraphBoxRegressorLightning().cuda()

# early stopping
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

early_stop_callback = EarlyStopping('val_huber_loss', patience = 30, mode = 'min')
checkpoint_callback = ModelCheckpoint(monitor = 'val_huber_loss', save_top_k = 1, mode = 'min', dirpath='/content/drive/MyDrive/Colab Notebooks/dl/graphattention/', filename='model-gam_l_huber-{epoch:02d}-{val_loss:.2f}')
# set up the trainer

trainer = pl.Trainer(accelerator='auto', max_epochs = 200 , callbacks=[early_stop_callback, checkpoint_callback])

# train the model
trainer.fit(model, train_loader, val_loader)



INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type              | Params
--------------------------------------------
0 | gam_l | GraphBoxRegressor | 13.1 M
1 | MSE   | MSELoss           | 0     
2 | MAE   | L1Loss            | 0     
3 | HUBER | SmoothL1Loss      | 0     
--------------------------------------------
13.1 M    Trainable params
0         Non-trainable params
13.1 M    Total params
52.418    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [12]:
!cp -r /content/lightning_logs /content/drive/MyDrive/Colab Notebooks/dl/graphattention/gam_l_log_huber/

cp: target 'Notebooks/dl/graphattention/gam_l_log_huber/' is not a directory
